# System performance analysis

## 1. Database connection
Establish connection to neo4j database with performance data.

In [141]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')


## 2. Cypher query
A cypher query is executed and the result is stored in a pandas dataframe.

In [142]:
import pandas as pd

# WHERE n.fileName =~ '.*/1-MemoryLeak-5/kieker-logs/kieker-20150820-064855519-UTC-middletier2-KIEKER'
query ="""
MATCH (n:Directory { fileName: 'D:/Users/Tom/dev/jqassistant-commandline-neo4jv3-1.8.0/MonitoringData/1-MemoryLeak-5/kieker-logs/kieker-20150820-064855519-UTC-middletier2-KIEKER' })-[:CONTAINS]->(c:CpuUtilizationRecord)
RETURN c.timestamp, c.hostname, c.cpuID, c.totalUtilization
ORDER BY c.timestamp
"""
df = pd.DataFrame(graph.run(query).data())
df = df.rename(columns={"c.timestamp": "Timestamp",
                        "c.hostname": "Hostname",
                        "c.cpuID": "CPU ID",
                        "c.totalUtilization": "CPU Utilization"})

## 3. Analaysis and Visualization
The data is analyzed and visualized.

In [143]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['CPU ID'] = pd.to_numeric(df['CPU ID'])

In [144]:
df.head()

,Timestamp,Hostname,CPU ID,CPU Utilization
0,2015-08-20 06:48:56.119231206,middletier2,0,0.900000
1,2015-08-20 06:48:56.119479386,middletier2,1,0.980392
2,2015-08-20 06:49:05.612040829,middletier2,0,0.128828
3,2015-08-20 06:49:05.612101780,middletier2,1,0.242616
4,2015-08-20 06:49:15.611996675,middletier2,0,0.036000


In [145]:
from IPython.display import display, HTML

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [146]:
import pygal as pg

line_chart = pg.Line()
line_chart.title = 'CPU Utilization' 
id0 = df.loc[df['CPU ID'] == 0]
# id1 = df.loc[df['CPU ID'] == 1]
line_chart.add('Core 0', pd.Series(id0['CPU Utilization']))
# line_chart.add('Core 1', pd.Series(id1['CPU Utilization']))

display(HTML(base_html.format(rendered_chart=line_chart.render(is_unicode=True))))